
## Segmenting and Clustering Neighborhoods in Toronto

## Introduction

The objective of this notebook is to cluster the neighborhoods of Toronto through the different venues and visualize using Folium tool. 
We will focus on the 4 Boroughs that have Toronto in their name. Let's call them Toronto "ceneter".
We will convert postal codes into their equivalent latitude and longitude values. 
Also, we will use the Foursquare API to explore neighborhoods in Toronto using the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. 
I will use the *k*-means clustering algorithm to complete this task. 
Finally, we will use the Folium library to visualize the clustered neighborhoods in Toronto City center.

## Start part 1. Toronto Neighboughoods



There are 4 boroughs with Toronto in his name that contain 39 neighborhoods. 
In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 4 "Toronto" boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each postal code. In the same postal code could be more than one neighborhood

Import needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests
from urllib.request import urlopen

import re

!pip install bs4
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


Reading a HTML page  using BeautifulSoup. 

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html=urlopen(url)
        
soup = BeautifulSoup(html,'html.parser')

Format the data using regular expressions and load it in a DataFrame

In [3]:
# define the dataframe columns
column_names = ['Postal Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
df_neighborhoods_toronto = pd.DataFrame(columns=column_names)

x=str(soup(class_='wikitable sortable'))
y=re.sub("\</tr>","&&&",x)
y=re.sub("\<tr>","",y)
y=re.sub("\</td>","",y)
y=re.sub("\<td>","",y)
y=re.sub("\<tr>","&&&",y)
y=y.split("&&&")
len(y)

# let's loop through the data and fill the dataframe one row at a time

for i in range(1,len(y)):
    r=re.sub("\\n\n","\\n",y[i])
    r=r.split("\n")
    
    try:
        df_neighborhoods_toronto.at[i-1,'Postal Code'] = r[1]
        df_neighborhoods_toronto.at[i-1,'Borough'] = r[2]
        df_neighborhoods_toronto.at[i-1,'Neighborhood'] = r[3]
    except:
        continue

# Show first 10 rows of the dataframe
df_neighborhoods_toronto.head(10)     

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Wrangling the data in the dataframe: 
    - Deleting rows without Borough and Neighborhood ("Not Assigned")
    - Asigning Neighborhood (same the Borough) if "Not assigned"

In [4]:
cond=(df_neighborhoods_toronto['Borough']!='Not assigned')
cond
df_neighborhoods_toronto1=df_neighborhoods_toronto[cond]
cond2=(df_neighborhoods_toronto1['Neighborhood']=='Not assigned')
df_neighborhoods_toronto1['Neighborhood'][cond2]=df_neighborhoods_toronto1['Borough']
#df_neighborhoods_toronto1.reset_index   
df_neighborhoods_toronto1.reset_index(drop=True,inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


## Results part 1. 

In [5]:
# print number of neighborhoods and the first 10 rows of the dataframe
print("number of neighborhoods: ", df_neighborhoods_toronto1.shape[0])
print(df_neighborhoods_toronto1.head(10))
# print(df_neighborhoods_toronto1.iloc[0][0])

number of neighborhoods:  103
  Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
5         M9A         Etobicoke      Islington Avenue, Humber Valley Village
6         M1B       Scarborough                               Malvern, Rouge
7         M3B        North York                                    Don Mills
8         M4B         East York              Parkview Hill, Woodbine Gardens
9         M5B  Downtown Toronto                     Garden District, Ryerson


## Start part 2. 

Unload csv file with the latitude and longitude of Toronto postal codes and load them in a dataframe

In [6]:
!wget -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
df_latlon = pd.read_csv('Geospatial_data.csv')
df_latlon.head()

--2020-06-10 17:17:46--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.24, 159.8.69.21, 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.69.24|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-06-10 17:17:46--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-10 17:17:47--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-10 17:17:48--  https://ibm.box.com/public/static/

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Obtain a dataframe with postal code, boroughs, neighborhood, latitude and longitude merging 2 dataframes 

In [7]:
# print number of neighborhoods and the first 10 rows of the dataframe with latitude and longitude
df_neighborhoods_toronto2=df_neighborhoods_toronto1.merge(df_latlon, left_on='Postal Code', right_on='Postal Code', how='inner')

print("Pre-selected {} neighborhoods in the {} Boroughs of all Toronto: ".format(df_neighborhoods_toronto2.shape[0],len(df_neighborhoods_toronto2['Borough'].unique())))

df_neighborhoods_toronto2.head(10)

Pre-selected 103 neighborhoods in the 10 Boroughs of all Toronto: 


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Selecting only the boroughs that has "Toronto" word in the name 

In [8]:
borough=set(df_neighborhoods_toronto2['Borough']) #get unique values of Borough
borough

{'Central Toronto',
 'Downtown Toronto',
 'East Toronto',
 'East York',
 'Etobicoke',
 'Mississauga',
 'North York',
 'Scarborough',
 'West Toronto',
 'York'}

## Results part 2. 

In [9]:
boroug_toronto=['Central Toronto','Downtown Toronto','East Toronto','West Toronto']
cond=df_neighborhoods_toronto2['Borough'].isin(boroug_toronto)
neighborhoods=df_neighborhoods_toronto2[cond]
print("There area {} group of neighborhoods in the {} Boroughs of Toronto center: ".format(neighborhoods.shape[0],len(boroug_toronto)))
neighborhoods.reset_index(drop=True,inplace=True)  
neighborhoods

There area 39 group of neighborhoods in the 4 Boroughs of Toronto center: 


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


## Start part 3. 

Import additional libraries needed

In [10]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

In [11]:
neighborhoods.sort_values('Borough',axis=0, inplace=True)
neighborhoods.reset_index(drop=True,inplace=True)
neighborhoods.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M5P,Central Toronto,"Forest Hill North &amp; West, Forest Hill Road...",43.696948,-79.411307
3,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
5,M4S,Central Toronto,Davisville,43.704324,-79.388790
6,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
7,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
8,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
9,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049


## 3.1. Getting the latitude and longitude values of Toronto City postal codes which group the neighborhoods (using geopy library

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [12]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and cluster them.

#### Define Foursquare Credentials and Version

In [52]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [15]:
neighborhoods.loc[0, 'Neighborhood']

'Roselawn'

We will do the clustering by postal code, which can include more than one neighborhood

In [16]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name
borough_name = neighborhoods.loc[0, 'Borough'] # borough name
codpost_name = neighborhoods.loc[0, 'Postal Code'] # borough name

print('Latitude and longitude values of {} in {} with postal code {} are {}, {}.'.format(neighborhood_name, 
                                                                     borough_name, 
                                                                     codpost_name,
                                                                     neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Roselawn in Central Toronto with postal code M5N are 43.7116948, -79.41693559999999.


#### Now, let's get the top 100 venues that are in Roselawn within a radius of 1000 meters, because using 500 meters were not enough for some postal codes with a few venues in.   

First, let's create the GET request URL. Name your URL **url**.

In [54]:

radius=1000
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude,neighborhood_longitude, VERSION, radius, LIMIT)




Send the GET request and examine the results

In [18]:
results = requests.get(url).json()
results
# results['response'].keys()

{'meta': {'code': 200, 'requestId': '5ee114471c213b759816a0bf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lawrence Park South',
  'headerFullLocation': 'Lawrence Park South, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.72069480900001,
    'lng': -79.40450770787952},
   'sw': {'lat': 43.702694790999985, 'lng': -79.42936349212046}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4fdc0e98e4b05197cd14912b',
       'name': 'The Abbot',
       'location': {'address': '508 Eglinton Ave W',
        'crossStreet': 'Heddington Ave',
        'lat': 43.703687730512435,
        'lng': -79.41348481516249,
        'l

We know (from the Foursquare lab) that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [20]:
results['response']['groups'][0]['items'][2]['venue']['categories']


[{'id': '4bf58dd8d48988d1e0931735',
  'name': 'Coffee Shop',
  'pluralName': 'Coffee Shops',
  'shortName': 'Coffee Shop',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
   'suffix': '.png'},
  'primary': True}]

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:,filtered_columns ]
print(nearby_venues.loc[0:4, ('venue.name', 'venue.categories')])

                  venue.name  \
0                  The Abbot   
1               Hotel Gelato   
2  The Mad Bean Coffee House   
3                  7 Numbers   
4                    Ferraro   

                                    venue.categories  
0  [{'id': '4bf58dd8d48988d155941735', 'name': 'G...  
1  [{'id': '4bf58dd8d48988d16d941735', 'name': 'C...  
2  [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...  
3  [{'id': '4bf58dd8d48988d110941735', 'name': 'I...  
4  [{'id': '4bf58dd8d48988d110941735', 'name': 'I...  


#### Let's create a function to repeat the same process to all  postal codes in Toronto "center"

In [22]:
def getNearbyVenues(postalcodes, names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for postalcode, name, lat, lng in zip(postalcodes, names, latitudes, longitudes):
        print(postalcode,name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postalcode,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Neighborhood', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# getting the category for each row cleaning the categorie column with the function nearby_venues.apply
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


In [24]:
print(nearby_venues.loc[0:4,'venue.categories' ])

0             Gastropub
1                  Café
2           Coffee Shop
3    Italian Restaurant
4    Italian Restaurant
Name: venue.categories, dtype: object


In [25]:
# clean columns getting only the last part of the name of each column
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Abbot,Gastropub,43.703688,-79.413485
1,Hotel Gelato,Café,43.703478,-79.414311
2,The Mad Bean Coffee House,Coffee Shop,43.703529,-79.413698
3,7 Numbers,Italian Restaurant,43.703630,-79.413724
4,Ferraro,Italian Restaurant,43.703655,-79.413167


And how many venues were returned by Foursquare?

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


<a id='item2'></a>

## 3.2. Explore Neighborhoods group by postal code in 4 Boroughs of Toronto "center"

#### Run the getNearbyVenues function on each postal code and load results into  a new dataframe called *toronto_venues*.

In [27]:
# type your answer here

toronto_venues = getNearbyVenues(postalcodes=neighborhoods['Postal Code'],
                                   names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



M5N Roselawn
M4P Davisville North
M5P Forest Hill North &amp; West, Forest Hill Road Park
M4R North Toronto West,  Lawrence Park
M5R The Annex, North Midtown, Yorkville
M4S Davisville
M4T Moore Park, Summerhill East
M4V Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
M4N Lawrence Park
M5T Kensington Market, Chinatown, Grange Park
M4Y Church and Wellesley
M5S University of Toronto, Harbord
M5V CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M4W Rosedale
M5W Stn A PO Boxes
M4X St. James Town, Cabbagetown
M5X First Canadian Place, Underground city
M5A Regent Park, Harbourfront
M5L Commerce Court, Victoria Hotel
M7A Queen's Park, Ontario Provincial Government
M5B Garden District, Ryerson
M5K Toronto Dominion Centre, Design Exchange
M5C St. James Town
M5E Berczy Park
M5G Central Bay Street
M5J Harbourfront East, Union Station, Toronto Islands
M6G Christie
M5H Richmond, Adelaide, King
M4E The Beaches
M7Y Business re

#### Let's check the size of the resulting dataframe

In [28]:
print(toronto_venues.shape)
toronto_venues.head(10)

(3199, 8)


,Postal Code,Neighborhood,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5N,Roselawn,43.711695,-79.416936,The Abbot,43.703688,-79.413485,Gastropub
1,M5N,Roselawn,43.711695,-79.416936,Hotel Gelato,43.703478,-79.414311,Café
2,M5N,Roselawn,43.711695,-79.416936,The Mad Bean Coffee House,43.703529,-79.413698,Coffee Shop
3,M5N,Roselawn,43.711695,-79.416936,7 Numbers,43.703630,-79.413724,Italian Restaurant
4,M5N,Roselawn,43.711695,-79.416936,Ferraro,43.703655,-79.413167,Italian Restaurant
5,M5N,Roselawn,43.711695,-79.416936,Tokyo Sushi,43.704146,-79.410631,Sushi Restaurant
6,M5N,Roselawn,43.711695,-79.416936,Phipps Bakery Cafe,43.704116,-79.411135,Bakery
7,M5N,Roselawn,43.711695,-79.416936,EDO,43.703754,-79.412802,Japanese Restaurant
8,M5N,Roselawn,43.711695,-79.416936,Forest Hill Arena,43.704289,-79.420367,Skating Rink
9,M5N,Roselawn,43.711695,-79.416936,Starbucks,43.704171,-79.411887,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Berczy Park,100,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",48,48,48,48,48,48,48
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15,15
Central Bay Street,100,100,100,100,100,100,100
Christie,100,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100
Davisville,100,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 288 uniques categories.


In [31]:
set(toronto_venues.loc[0:10,'Venue Category'])

{'Bakery',
 'Bank',
 'Café',
 'Coffee Shop',
 'Gastropub',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Skating Rink',
 'Sushi Restaurant'}

<a id='item3'></a>

## 3.3. Analyze Each Neighborhood

In [32]:
# one hot encoding
toronto_onehot=[]

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Resta

In [33]:
# add neighborhood column back to dataframe
toronto_onehot['tNeighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column. I renamed with "tNeighborhood because there is another column with this name"
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,tNeighborhood,Postal Code,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian

In [34]:
cond=toronto_onehot['Neighborhood']!=0
toronto_onehot[(cond)]

,tNeighborhood,Postal Code,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian

And let's examine the new dataframe size.

In [35]:
toronto_onehot.shape

(3199, 290)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head(5)

,Postal Code,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Resta

#### Let's confirm the new size

In [37]:
toronto_grouped.shape

(39, 289)

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                 venue  freq
0                  Pub  0.06
1          Coffee Shop  0.06
2          Pizza Place  0.04
3       Breakfast Spot  0.04
4  Japanese Restaurant  0.04


----M4K----
                venue  freq
0    Greek Restaurant  0.11
1                Café  0.06
2         Coffee Shop  0.06
3                 Pub  0.05
4  Italian Restaurant  0.03


----M4L----
                  venue  freq
0     Indian Restaurant  0.11
1           Coffee Shop  0.05
2            Restaurant  0.04
3                 Beach  0.04
4  Fast Food Restaurant  0.04


----M4M----
                 venue  freq
0          Coffee Shop  0.07
1                  Bar  0.06
2              Brewery  0.05
3                 Café  0.05
4  American Restaurant  0.04


----M4N----
                  venue  freq
0            Restaurant  0.11
1             Bookstore  0.11
2  Gym / Fitness Center  0.11
3                 Trail  0.11
4           Coffee Shop  0.11


----M4P----
                venue  freq
0         Coff

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Now let's create the new dataframe and display the top 10 venues for each postal code  / group of neighborhoods.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Coffee Shop,Pub,Japanese Restaurant,Pizza Place,Breakfast Spot,Beach,Bakery,Indian Restaurant,Park,Caribbean Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Café,Pub,Fast Food Restaurant,Pizza Place,Italian Restaurant,Bank,Discount Store,Yoga Studio
2,M4L,Indian Restaurant,Coffee Shop,Café,Fast Food Restaurant,Restaurant,Beach,Brewery,Sandwich Place,Grocery Store,Bakery
3,M4M,Coffee Shop,Bar,Brewery,Café,Diner,American Restaurant,Vietnamese Restaurant,Bakery,Italian Restaurant,French Restaurant
4,M4N,Gym / Fitness Center,Trail,Park,Bookstore,College Quad,Restaurant,College Gym,Coffee Shop,Café,Donut Shop


<a id='item4'></a>

## 3.4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [42]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 2, 2, 2, 0, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
# add clustering labels if not exists (if I ran before the cell)
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    neighborhoods_venues_sorted['Cluster Labels']=kmeans.labels_    

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude and neighborhood for each postal code
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged.head(10) # check the first rows!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Café,Bank,Bagel Shop,Japanese Restaurant,Lingerie Store,Asian Restaurant
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Coffee Shop,Italian Restaurant,Café,Pizza Place,Restaurant,Fast Food Restaurant,Dessert Shop,Pharmacy,Gym,Sushi Restaurant
2,M5P,Central Toronto,"Forest Hill North &amp; West, Forest Hill Road...",43.696948,-79.411307,2,Park,Coffee Shop,Bank,Café,Skating Rink,Italian Restaurant,Japanese Restaurant,Gym / Fitness Center,Burger Joint,Pharmacy
3,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,2,Coffee Shop,Italian Restaurant,Park,Mexican Restaurant,Skating Rink,Restaurant,Sporting Goods Shop,Diner,Café,Chinese Restaurant
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Gym,Pizza Place,Museum
5,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Gym,Restaurant,Indian Restaurant,Dessert Shop,Pub
6,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Coffee Shop,Grocery Store,Italian Restaurant,Park,Thai Restaurant,Gym,Bagel Shop,Restaurant,Bank,Burger Joint
7,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,2,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Sandwich Place,Café,Bagel Shop,Gym
8,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Gym / Fitness Center,Trail,Park,Bookstore,College Quad,Restaurant,College Gym,Coffee Shop,Café,Donut Shop
9,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,1,Café,Bar,Vegetarian / Vegan Restaurant,Art Gallery,Yoga Studio,Coffee Shop,Mexican Restaurant,Arts & Crafts Store,Record Shop,Gym


Finally, let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 3.5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, I assigned a category to each cluster (in capital letters) 

#### Cluster 0:
RED (in the map): 
**RESIDENTIAL & COMMERCIAL AREA:** Restaurants, cafes, bakeries, stores


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1]+ [2]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,"Moore Park, Summerhill East",0,Coffee Shop,Grocery Store,Italian Restaurant,Park,Thai Restaurant,Gym,Bagel Shop,Restaurant,Bank,Burger Joint
19,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Park,Japanese Restaurant,Café,Burger Joint,Ramen Restaurant,Middle Eastern Restaurant,Pizza Place,Clothing Store
28,East Toronto,The Beaches,0,Coffee Shop,Pub,Japanese Restaurant,Pizza Place,Breakfast Spot,Beach,Bakery,Indian Restaurant,Park,Caribbean Restaurant
29,East Toronto,"Business reply mail Processing Centre, South C...",0,Park,Brewery,Pizza Place,Coffee Shop,Italian Restaurant,Bakery,Sushi Restaurant,Fast Food Restaurant,Burrito Place,Electronics Store
30,East Toronto,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Café,Pub,Fast Food Restaurant,Pizza Place,Italian Restaurant,Bank,Discount Store,Yoga Studio
31,East Toronto,"India Bazaar, The Beaches West",0,Indian Restaurant,Coffee Shop,Café,Fast Food Restaurant,Restaurant,Beach,Brewery,Sandwich Place,Grocery Store,Bakery
32,East Toronto,Studio District,0,Coffee Shop,Bar,Brewery,Café,Diner,American Restaurant,Vietnamese Restaurant,Bakery,Italian Restaurant,French Restaurant
33,West Toronto,"Dufferin, Dovercourt Village",0,Café,Coffee Shop,Park,Italian Restaurant,Sushi Restaurant,Bar,Bakery,Restaurant,Portuguese Restaurant,Gourmet Shop
34,West Toronto,"Parkdale, Roncesvalles",0,Pizza Place,Coffee Shop,Sushi Restaurant,Breakfast Spot,Bar,Café,Thai Restaurant,Bakery,Eastern European Restaurant,Restaurant
36,West Toronto,"High Park, The Junction South",0,Café,Bar,Coffee Shop,Thai Restaurant,Italian Restaurant,Convenience Store,Park,Sushi Restaurant,Mexican Restaurant,Pharmacy


#### Cluster 1
VIOLET (in the map): 
**TURISTIC AND CULTURAL AREA:** Hotels, theaters, concert hall, cafes, record shops, restaurants

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1]+ [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,"The Annex, North Midtown, Yorkville",1,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Gym,Pizza Place,Museum
9,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",1,Café,Bar,Vegetarian / Vegan Restaurant,Art Gallery,Yoga Studio,Coffee Shop,Mexican Restaurant,Arts & Crafts Store,Record Shop,Gym
10,Downtown Toronto,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Burger Joint,Restaurant,Park,Sushi Restaurant,Bubble Tea Shop,Café,Caribbean Restaurant,Bookstore
11,Downtown Toronto,"University of Toronto, Harbord",1,Café,Bakery,Bar,Coffee Shop,Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Bookstore,Pizza Place,Arts & Crafts Store
14,Downtown Toronto,Stn A PO Boxes,1,Coffee Shop,Café,Japanese Restaurant,Restaurant,Hotel,Park,Seafood Restaurant,Cocktail Bar,Gastropub,Creperie
15,Downtown Toronto,"St. James Town, Cabbagetown",1,Diner,Park,Japanese Restaurant,Restaurant,Gastropub,Café,Pet Store,Botanical Garden,Steakhouse,Sushi Restaurant
16,Downtown Toronto,"First Canadian Place, Underground city",1,Coffee Shop,Hotel,Café,Theater,Restaurant,Japanese Restaurant,Concert Hall,Thai Restaurant,Park,Seafood Restaurant
18,Downtown Toronto,"Commerce Court, Victoria Hotel",1,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gastropub,Seafood Restaurant,Theater,Bakery,Concert Hall
20,Downtown Toronto,"Garden District, Ryerson",1,Coffee Shop,Gastropub,Japanese Restaurant,Theater,Hotel,Café,Creperie,Seafood Restaurant,Italian Restaurant,Restaurant
21,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",1,Coffee Shop,Café,Hotel,Japanese Restaurant,Theater,Restaurant,Concert Hall,Deli / Bodega,Cocktail Bar,Park


#### Cluster 2
BLUE (in the map): 
**RESIDENTIAL & SPORT AREA:** Parks, Gym, skating rings, cafes and restaurants



In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1]+ [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Roselawn,2,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pharmacy,Café,Bank,Bagel Shop,Japanese Restaurant,Lingerie Store,Asian Restaurant
1,Central Toronto,Davisville North,2,Coffee Shop,Italian Restaurant,Café,Pizza Place,Restaurant,Fast Food Restaurant,Dessert Shop,Pharmacy,Gym,Sushi Restaurant
2,Central Toronto,"Forest Hill North &amp; West, Forest Hill Road...",2,Park,Coffee Shop,Bank,Café,Skating Rink,Italian Restaurant,Japanese Restaurant,Gym / Fitness Center,Burger Joint,Pharmacy
3,Central Toronto,"North Toronto West, Lawrence Park",2,Coffee Shop,Italian Restaurant,Park,Mexican Restaurant,Skating Rink,Restaurant,Sporting Goods Shop,Diner,Café,Chinese Restaurant
5,Central Toronto,Davisville,2,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Gym,Restaurant,Indian Restaurant,Dessert Shop,Pub
7,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",2,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Sandwich Place,Café,Bagel Shop,Gym
17,Downtown Toronto,"Regent Park, Harbourfront",2,Coffee Shop,Diner,Café,Theater,Park,Pub,Restaurant,Breakfast Spot,Bakery,Italian Restaurant
38,West Toronto,"Runnymede, Swansea",2,Coffee Shop,Café,Pizza Place,Bakery,Sushi Restaurant,Italian Restaurant,Restaurant,Pub,Bank,Falafel Restaurant


#### Cluster 3
SKY BLUE (in the map): 
**EDUCATIONAL & OPEN-AIR LEISURE:** Parks, trails, colleges and bookstore


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + [1]+list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Lawrence Park,Central Toronto,3,Gym / Fitness Center,Trail,Park,Bookstore,College Quad,Restaurant,College Gym,Coffee Shop,Café,Donut Shop


#### Cluster 4
Green (in the map): 
**WORK AND SPORT AREA**. Office, breakfast spot, park, athletics and sports

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2]+[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Rosedale,Downtown Toronto,4,Coffee Shop,Grocery Store,Park,Athletics & Sports,Filipino Restaurant,BBQ Joint,Candy Store,Japanese Restaurant,Office,Breakfast Spot


#### Cluster 5
Orange (in the map):
**TRANSPORT HUB, LEISURE AND NIGHT LIFE:** Harbour, airport. Parks, garden and trails. Dance studio and coffe shop

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2]+[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"CN Tower, King and Spadina, Railway Lands, Har...",Downtown Toronto,5,Coffee Shop,Café,Harbor / Marina,Dog Run,Scenic Lookout,Garden,Park,Track,Sculpture Garden,Dance Studio
